# Scenario 2 : interaction case

## library

In [97]:
import pandas as pd
import numpy as np
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.conversion import localconverter

import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
from rpy2.robjects import numpy2ri
from rpy2.robjects.vectors import StrVector

required_packages = ['base', 'forecast', 'CondIndTests','RCIT'] # list of required R packages 
if all(rpackages.isinstalled(x) for x in required_packages):
    check_packages = True # True if packages are already installed 
else:
   check_packages = False # False if packages are not installed 
if check_packages == False: # Not installed? Then install.
    utils = rpackages.importr('utils')
    utils.chooseCRANmirror(ind=1)
    packages_to_install = [x for x in required_packages if not rpackages.isinstalled(x)]
    if len(packages_to_install) > 0:
        utils.install_packages(StrVector(packages_to_install))
    check_packages = True 

In [98]:
r = robjects.r
base = importr('base')
forecast = importr('forecast')
graphics = importr('graphics')
grdevices = importr('grDevices')
condindtests = importr('CondIndTests')

In [99]:
RCIT = importr('RCIT')

In [100]:
pcalg = importr('pcalg')

In [101]:
def IgnorTest(cause=None, effect=None, explorer=None, covariate=None, level=0.05, dtype="continuous"): #inputs are arrays, dtypes are continuous/binary/mixed
    tmp_cause=pd.DataFrame({'A': cause})
    tmp_effect=pd.DataFrame({'Y': effect})
    tmp_explorer=pd.DataFrame({'V': explorer})
    tmp_covariate = pd.DataFrame(covariate)
    dim_covariate = tmp_covariate.shape[1]
    
    if dtype=="continuous":
        if dim_covariate==0:
            tmp_conditioning=tmp_cause
        else:
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)

        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_conditioning = ro.conversion.py2rpy(tmp_conditioning)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_effect = ro.conversion.py2rpy(tmp_effect)
        with localconverter(ro.default_converter + pandas2ri.converter):
            r_from_pd_explorer = ro.conversion.py2rpy(tmp_explorer)
            
        sample_size = r.unlist(r.dim(r_from_pd_conditioning))[0]
        dim = r.unlist(r.dim(r_from_pd_conditioning))[1]
            
        r_from_pd_explorer=r.unlist(r_from_pd_explorer)    
        r_from_pd_effect=r.unlist(r_from_pd_effect)   
        r_from_pd_conditioning=r.unlist(r_from_pd_conditioning)   
        r_from_pd_conditioning=r.matrix(r_from_pd_conditioning,sample_size,dim)
        
        #test=r.CondIndTest(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning, method = "KCI")
        #pvalue=r.unlist(test)[3]
        
        test=r.RCIT(r_from_pd_explorer,r_from_pd_effect,r_from_pd_conditioning,seed=2)
        pvalue=r.unlist(test)[0]
        print(pvalue)
           
    elif dtype=="binary":
        if covariate==None:
            tmp_conditioning=tmp_cause
        else:
            tmp_covariate=pd.DataFrame(covariate)
            tmp_conditioning=pd.concat([tmp_cause,tmp_covariate],axis=1)
            
    elif dtype=="mixed":
        pass
                        
    else:
        raise ValueError("explorertype is not valid.")
        
    if(pvalue<=level):
        print("The Ignorability Assumption is not satisfied.")
    else:
        print("The Ignorability Assumption is satisfied.")

    return pvalue

### parameter setting

In [102]:
n = 4000 #changeable
c1 = 1
c2 = 1 
c3 = 1 #changeable
c4 = 1
c5 = 1
r1 = r2 = r3 = r4 = 1 #changeable
import scipy.stats as stats

### Simulation

In [103]:
#Figure 1
np.random.seed(1)

COUNT1=0 #기각 횟수
for i in range(1000):

    U = np.random.normal(0,1,n) #common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+ey
    
    if i==1:
        f1_cor12=stats.pearsonr(X1, X2)[0]
    
    pvalue = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous")
    if pvalue<=0.05:
        COUNT1 = COUNT1+1

0.3616639405295883
The Ignorability Assumption is satisfied.
0.9071765366257789
The Ignorability Assumption is satisfied.
0.5467693563011407
The Ignorability Assumption is satisfied.
0.10138589743870852
The Ignorability Assumption is satisfied.
0.31689857966910373
The Ignorability Assumption is satisfied.
0.15888946829369577
The Ignorability Assumption is satisfied.
0.6263184804027031
The Ignorability Assumption is satisfied.
0.9543854471516661
The Ignorability Assumption is satisfied.
0.6625853345895432
The Ignorability Assumption is satisfied.
0.31604380090257667
The Ignorability Assumption is satisfied.
0.9294813547852536
The Ignorability Assumption is satisfied.
0.3420265221088904
The Ignorability Assumption is satisfied.
0.9524158090988092
The Ignorability Assumption is satisfied.
0.3496629323761584
The Ignorability Assumption is satisfied.
0.5533453266723067
The Ignorability Assumption is satisfied.
0.04521695983573237
The Ignorability Assumption is not satisfied.
0.0043586660618

In [104]:
#Figure 2
np.random.seed(1)

COUNT21=0 #기각 횟수
COUNT22=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f2_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT21 = COUNT21+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT22 = COUNT22+1 #2종 오류

0.9121047839829974
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.6611961466431652
The Ignorability Assumption is satisfied.
2.886579864025407e-15
The Ignorability Assumption is not satisfied.
0.7066091206073606
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.6845530988591437
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.20235434986509093
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.44898440845087717
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.9949257306367949
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7739614954292886
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.921655085128913
T

In [105]:
#Figure 3
np.random.seed(1)

COUNT31=0 #기각 횟수
COUNT32=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    Y = c3*C+ey
    
    if i==1:
        f3_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT31 = COUNT31+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT32 = COUNT32+1 #2종 오류

0.47111107595377266
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7210101493007999
The Ignorability Assumption is satisfied.
2.4424906541753444e-15
The Ignorability Assumption is not satisfied.
0.19952460366227776
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7921609439460501
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.10513595780423812
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.48126637171723496
The Ignorability Assumption is satisfied.
4.440892098500626e-16
The Ignorability Assumption is not satisfied.
0.97514080531661
The Ignorability Assumption is satisfied.
1.4432899320127035e-15
The Ignorability Assumption is not satisfied.
0.7698033266872486
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8862603501306663
The Ignorability Assumption is sati

In [106]:
#Figure 4 
np.random.seed(1)

COUNT41=0 #기각 횟수
COUNT42=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c4*X2+c3*C+X1*X2+X1*C+X2*C+ey
    
    if i==1:
        f4_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT41 = COUNT41+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT42 = COUNT42+1 #2종 오류

0.3339406101460064
The Ignorability Assumption is satisfied.
3.4059937870090096e-08
The Ignorability Assumption is not satisfied.
0.3313963092890392
The Ignorability Assumption is satisfied.
1.8274428636999573e-09
The Ignorability Assumption is not satisfied.
0.5583545787452856
The Ignorability Assumption is satisfied.
7.052618177239012e-07
The Ignorability Assumption is not satisfied.
0.8916286992047038
The Ignorability Assumption is satisfied.
2.899319784255283e-09
The Ignorability Assumption is not satisfied.
0.8676881340189562
The Ignorability Assumption is satisfied.
7.157197057239273e-10
The Ignorability Assumption is not satisfied.
0.33125549834505097
The Ignorability Assumption is satisfied.
6.83720857708181e-11
The Ignorability Assumption is not satisfied.
0.6575691250508091
The Ignorability Assumption is satisfied.
0.012371675760750489
The Ignorability Assumption is not satisfied.
0.5258526070227301
The Ignorability Assumption is satisfied.
1.010652161959058e-08
The Ignorabil

In [107]:
#Figure 5
np.random.seed(1)

COUNT51=0 #기각 횟수
COUNT52=0
COUNT53=0
COUNT54=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f5_cor12=r1*r2/np.sqrt(r1**2+c2**2+1)/np.sqrt(r2**2+1)
        f5_cor13=r1*r3/np.sqrt(r1**2+c2**2+1)/np.sqrt(r3**2+1)
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT51 = COUNT51+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT52 = COUNT52+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT53 = COUNT53+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT54 = COUNT54+1 #2종 오류

0.022422362040416588
The Ignorability Assumption is not satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5912042791262537
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.13991019474470512
The Ignorability Assumption is satisfied.
1.1102230246251565e-15
The Ignorability Assumption is not satisfied.
0.818109789855689
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.2342936574751744
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.7251171012801996
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.39638340297394403
The Ignorability Assumption is satisfied.
5.551115123125783e-16
The Ignorability Assumption is not satisfied.
0.9526755610802057
The Ignorability Assumption is satisfied.
8.881784197001252e-16
The Ignorability Assumption is not satisfied.
0.9612649185309562
The Ignorabi

In [108]:
c3

1

In [109]:
print(f5_cor12)
print(f5_cor13)

0.408248290463863
0.408248290463863


In [110]:
COUNT51

47

In [111]:
COUNT52

1000

In [112]:
COUNT53

58

In [113]:
COUNT54

1000

In [114]:
#Figure 6
np.random.seed(1)

COUNT61=0 #기각 횟수
COUNT62=0
COUNT63=0
COUNT64=0
for i in range(1000):
    U1 = np.random.normal(0,1,n) #common cause of X1 and X2
    U2 = np.random.normal(0,1,n) #common cause of X1 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U1+c2*C+r3*U2+e1
    X2 = r2*U1+e2
    X3 = r4*U2+e3
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f6_cor12=r1*r2/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r2**2+1)
        f6_cor13=r3*r4/np.sqrt(r1**2+c2**2+r3**2+1)/np.sqrt(r4**2+1)

    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT61 = COUNT61+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT62 = COUNT62+1 #2종 오류
    pvalue3 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=C,dtype="continuous") 
    if pvalue3<=0.05:
        COUNT63 = COUNT63+1 #1종 오류
    pvalue4 = IgnorTest(cause=X1,effect=Y,explorer=X3,dtype="continuous") 
    if pvalue4<=0.05:
        COUNT64 = COUNT64+1 #2종 오류

0.936390745545393
The Ignorability Assumption is satisfied.
8.881784197001252e-16
The Ignorability Assumption is not satisfied.
0.25014803846147193
The Ignorability Assumption is satisfied.
2.5044540885588162e-09
The Ignorability Assumption is not satisfied.
0.675582152384562
The Ignorability Assumption is satisfied.
4.42758996133108e-09
The Ignorability Assumption is not satisfied.
0.907720674712486
The Ignorability Assumption is satisfied.
5.455306206769706e-10
The Ignorability Assumption is not satisfied.
0.9885384630113961
The Ignorability Assumption is satisfied.
1.9984014443252818e-15
The Ignorability Assumption is not satisfied.
0.2639507552600747
The Ignorability Assumption is satisfied.
3.482769628249116e-13
The Ignorability Assumption is not satisfied.
0.28583795230577014
The Ignorability Assumption is satisfied.
5.1064708017634075e-12
The Ignorability Assumption is not satisfied.
0.9504845674614304
The Ignorability Assumption is satisfied.
8.659739592076221e-15
The Ignorabil

In [115]:
print(f6_cor12)
print(f6_cor13)

0.35355339059327373
0.35355339059327373


In [116]:
COUNT61

45

In [117]:
COUNT62

1000

In [118]:
COUNT63

49

In [119]:
COUNT64

1000

In [120]:
#Figure 7
np.random.seed(1)

COUNT71=0 #기각 횟수
COUNT72=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder
    eu = np.random.normal(0,1,n)
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    U = c2*C+eu #common cause of X1 and X2
    X1 = r1*U+e1
    X2 = r2*U+e2
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f7_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT71 = COUNT71+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT72 = COUNT72+1 #2종 오류

0.8120476489756887
The Ignorability Assumption is satisfied.
1.5543122344752192e-15
The Ignorability Assumption is not satisfied.
0.8370030028899182
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6083265246837699
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5374668529970499
The Ignorability Assumption is satisfied.
9.992007221626409e-16
The Ignorability Assumption is not satisfied.
0.1644179133277589
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.24169280446534291
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.8196814424239834
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.16050699306111882
The Ignorability Assumption is satisfied.
7.771561172376096e-16
The Ignorability Assumption is not satisfied.
0.5451880529975527
The Ignorability 

In [121]:
#Figure 8
np.random.seed(1)

COUNT81=0 #기각 횟수
COUNT82=0
for i in range(1000):
    C = np.random.normal(0,1,n) #Confounder, common cause of X1 and X2
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*C+e1
    X2 = r2*C+e2
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f8_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT81 = COUNT81+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT82 = COUNT82+1 #2종 오류

0.27097195169170973
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.2901667098623607
The Ignorability Assumption is satisfied.
8.881784197001252e-16
The Ignorability Assumption is not satisfied.
0.7415324383319144
The Ignorability Assumption is satisfied.
2.220446049250313e-16
The Ignorability Assumption is not satisfied.
0.3823692370251759
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7916484701524115
The Ignorability Assumption is satisfied.
1.1102230246251565e-15
The Ignorability Assumption is not satisfied.
0.3531872246791201
The Ignorability Assumption is satisfied.
1.2212453270876722e-15
The Ignorability Assumption is not satisfied.
0.28971326029818323
The Ignorability Assumption is satisfied.
1.3322676295501878e-15
The Ignorability Assumption is not satisfied.
0.6964315198327907
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.

In [122]:
#Figure 9
np.random.seed(1)

COUNT91=0 #기각 횟수
COUNT92=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    Y = c1*X1+c3*C+X1*C+ey
    
    if i==1:
        f9_cor12=stats.pearsonr(X1, X2)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X2,covariate=C,dtype="continuous") 
    if pvalue1<=0.05:
        COUNT91 = COUNT91+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT92 = COUNT92+1 #2종 오류

0.9201751976481886
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.8140465240689516
The Ignorability Assumption is satisfied.
1.887379141862766e-15
The Ignorability Assumption is not satisfied.
0.8015774590387654
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8642874844194371
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.14573292843466878
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7600700408760788
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.9789187173635922
The Ignorability Assumption is satisfied.
6.661338147750939e-16
The Ignorability Assumption is not satisfied.
0.9090184066906035
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.8798714978710711
The Ignorability Ass

In [123]:
#Figure 10
np.random.seed(1)

COUNT101=0 #기각 횟수
COUNT102=0
for i in range(1000):
    U = np.random.normal(0,1,n) #common cause of X1 and X2 and X3
    C = np.random.normal(0,1,n) #Confounder
    e1 = np.random.normal(0,1,n)
    e2 = np.random.normal(0,1,n)
    e3 = np.random.normal(0,1,n)
    ey = np.random.normal(0,1,n)
    
    X1 = r1*U+c2*C+e1
    X2 = r2*U+c4*C+e2
    X3 = r3*U+e3
    Y = c1*X1+c3*C+c5*X2+X1*X2+X1*C+X2*C+ey
    
    if i==1:
        f10_cor13=stats.pearsonr(X1, X3)[0]
        
    pvalue1 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=np.column_stack((X2,C)),dtype="continuous") 
    if pvalue1<=0.05:
        COUNT101 = COUNT101+1 #1종 오류
    pvalue2 = IgnorTest(cause=X1,effect=Y,explorer=X3,covariate=X2,dtype="continuous") 
    if pvalue2<=0.05:
        COUNT102 = COUNT102+1 #2종 오류

0.2446208935882338
The Ignorability Assumption is satisfied.
1.1102230246251565e-16
The Ignorability Assumption is not satisfied.
0.20270451811042
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.6965247681928375
The Ignorability Assumption is satisfied.
9.992007221626409e-16
The Ignorability Assumption is not satisfied.
0.8214284831840108
The Ignorability Assumption is satisfied.
8.881784197001252e-16
The Ignorability Assumption is not satisfied.
0.7422521399949569
The Ignorability Assumption is satisfied.
9.992007221626409e-16
The Ignorability Assumption is not satisfied.
0.3502249254383458
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.7430091108993722
The Ignorability Assumption is satisfied.
2.7777780076121417e-13
The Ignorability Assumption is not satisfied.
0.743122269015398
The Ignorability Assumption is satisfied.
0.0
The Ignorability Assumption is not satisfied.
0.5116581850904887
The 

In [124]:
table1=pd.DataFrame(np.repeat(-1,70).reshape(10,7))
table1.columns = ["Scenario1","1종오류(X2)","2종오류(X2)","1종오류(X3)","2종오류(X3)","r12","r13"]
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,-1,-1,-1,-1,-1,-1,-1
1,-1,-1,-1,-1,-1,-1,-1
2,-1,-1,-1,-1,-1,-1,-1
3,-1,-1,-1,-1,-1,-1,-1
4,-1,-1,-1,-1,-1,-1,-1
5,-1,-1,-1,-1,-1,-1,-1
6,-1,-1,-1,-1,-1,-1,-1
7,-1,-1,-1,-1,-1,-1,-1
8,-1,-1,-1,-1,-1,-1,-1
9,-1,-1,-1,-1,-1,-1,-1


In [125]:
table1["Scenario1"]=["Figure"+str(i+1) for i in range(10)]

In [126]:
table1["1종오류(X2)"]=[COUNT1,COUNT21,COUNT31,COUNT41,COUNT51,COUNT61,COUNT71,COUNT81,COUNT91,COUNT101]
table1["2종오류(X2)"]=[-1,COUNT22,COUNT32,COUNT42,COUNT52,COUNT62,COUNT72,COUNT82,COUNT92,COUNT102]
table1["1종오류(X3)"]=[-1,-1,-1,-1,COUNT53,COUNT63,-1,-1,-1,-1]
table1["2종오류(X3)"]=[-1,-1,-1,-1,COUNT54,COUNT64,-1,-1,-1,-1]
table1["r12"]=[f1_cor12,f2_cor12,f3_cor12,-1,f5_cor12,f6_cor12,f7_cor12,f8_cor12,f9_cor12,-1]
table1["r13"]=[-1,-1,-1,f4_cor13,f5_cor13,f6_cor13,-1,-1,-1,f10_cor13]

In [127]:
table1

,Scenario1,1종오류(X2),2종오류(X2),1종오류(X3),2종오류(X3),r12,r13
0,Figure1,60,-1,-1,-1,0.499098,-1.000000
1,Figure2,53,1000,-1,-1,0.413865,-1.000000
2,Figure3,56,1000,-1,-1,0.413865,-1.000000
3,Figure4,61,999,-1,-1,-1.000000,0.396688
4,Figure5,47,1000,58,1000,0.408248,0.408248
5,Figure6,45,1000,49,1000,0.353553,0.353553
6,Figure7,56,1000,-1,-1,0.664412,-1.000000
7,Figure8,50,1000,-1,-1,0.499098,-1.000000
8,Figure9,61,1000,-1,-1,0.664412,-1.000000
9,Figure10,66,1000,-1,-1,-1.000000,0.396688


In [128]:
filename="INTERACTION_c"+str(c3)+"_r"+str(r1)+"_s"+str(n/1000)+".csv"
table1.to_csv("C:\\Users\\minsoo\\Desktop\\실험결과(P)\\"+filename,encoding="euc-kr",index=False)